In [1]:
%matplotlib inline
from stl import mesh
from mpl_toolkits import mplot3d
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import pyvista
import copy
import scipy.io

#### Task 1

In [25]:
def findBoard(imPath, board, criteria):
    cols = board[0]
    rows = board[1]
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(cols-1,rows-1,0)
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:cols,0:rows].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    foundImages = []
    images = os.listdir(imPath)
    print("ALL", images)
    for fname  in images:
        img = cv2.imread(os.path.join(imPath, fname))
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        shape = gray.shape[::-1]
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (cols,rows), None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv2.cornerSubPix(gray,corners, (21,21), (-1,-1), criteria)
            imgpoints.append(corners2)
            foundImages.append(fname)
            print(fname)
            # Draw and display the corners
            cv2.drawChessboardCorners(img, (cols,rows), corners2, ret)
            cv2.imwrite(os.path.join("chessboard", fname), img)

    
    print("FOUND", foundImages)
    
    return objpoints, imgpoints, shape, foundImages

In [26]:
def ZhangCalibration(objpoints, imgpoints, shape):
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)
    errors = []
    mean_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        errors.append(error)
        mean_error += error
    print( "total error: {}".format(mean_error/len(objpoints)))
    return errors, mtx, dist, rvecs, tvecs

In [27]:
def undistort(image, mtx, dist):
    h,  w = image.shape[:2]
    newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

    # undistort
    dst = cv2.undistort(image, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    return dst

In [28]:
def concat(im1, im2, scale=100):
    
    h1, w1 = im1.shape[:2]
    h2, w2 = im2.shape[:2]

    im3 = np.zeros((h1+h2, max(w1,w2),3), dtype=np.uint8)
    im3[:,:] = (0,0,0)

    im3[:h1, :w1,:3] = im1
    im3[h1:h1+h2, :w2,:3] = im2

    width = int(im3.shape[1] * scale / 100)
    height = int(im3.shape[0] * scale / 100)
    dim = (width, height)

    im3 = cv2.resize(im3, dim, interpolation = cv2.INTER_AREA)

    return im3

In [29]:
path1 = "part2_images1"
path2 = "part2_images2"
path3 = "part2_images3"

selectedPath = "selected"

path1Files = os.listdir(path1)
path2Files = os.listdir(path2)
path3Files = os.listdir(path3)
selectedPathFiles = os.listdir(selectedPath)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS, 30, 0.001)
board = (9,6)

In [30]:
for fname in selectedPathFiles:
    print(os.path.join(selectedPath, fname))

selected\IMG_4653.JPG
selected\IMG_4654.JPG
selected\IMG_4657.JPG
selected\IMG_4662.JPG
selected\IMG_4665.JPG
selected\IMG_4669.JPG


In [31]:
# selecteds
objpoints, imgpoints, shape, foundImages = findBoard(selectedPath, board, criteria)
errors, mtx, dist, rvecs, tvecs = ZhangCalibration(objpoints, imgpoints, shape)
for fname in foundImages:
    img = cv2.imread(os.path.join(selectedPath, fname))
    undistorted = undistort(img, mtx, dist)
    concated = concat(img, undistorted)
    cv2.imwrite(os.path.join("concated", fname), concated)
    cv2.imwrite(os.path.join("undistorted", fname), undistorted)

ALL ['IMG_4653.JPG', 'IMG_4654.JPG', 'IMG_4657.JPG', 'IMG_4662.JPG', 'IMG_4665.JPG', 'IMG_4669.JPG']
IMG_4653.JPG
IMG_4654.JPG
IMG_4657.JPG
IMG_4662.JPG
IMG_4665.JPG
IMG_4669.JPG
FOUND ['IMG_4653.JPG', 'IMG_4654.JPG', 'IMG_4657.JPG', 'IMG_4662.JPG', 'IMG_4665.JPG', 'IMG_4669.JPG']
total error: 0.11232097963046513


In [32]:
%matplotlib
fig, ax = plt.subplots()
ax.scatter(range(1, len(imgpoints)+1,1), errors)
ax.set_title("Reprojection Errors")

Using matplotlib backend: TkAgg


Text(0.5, 1.0, 'Reprojection Errors')

#### Task 2

In [33]:
def projectCircle(img, imPoints, color=[0, 255, 0], radius = 1):
    rows,cols, _ = img.shape
    dst = img.copy()
    for i in range(imPoints.shape[0]):
        x = int(imPoints[i][0])
        y = int(imPoints[i][1])
        if(y < rows and y > 0 and x < cols and x > 0):
            dst = cv2.circle(dst, (x,y), radius=radius,  color=color)
            # dst[x, y] = np.array(color)
            # print(x,y, dst[x, y])

    return dst

def putModelonBoard(mesh, x = 0, y = 0, scale = 1):
    newMesh = copy.copy(mesh)
    xmin, xmax, ymin, ymax, zmin, zmax = mesh.bounds
    maxAxis = max(list(map(lambda x: abs(x), mesh.bounds)))
    for i in range(mesh.points.shape[0]):
        newX = scale*(mesh.points[i][0] - xmin) + x
        newY = scale*(mesh.points[i][1] - ymin) + y
        newZ = scale*(mesh.points[i][2] - zmin)
        newMesh.points[i] = [newX, newY, newZ]
    return newMesh

In [34]:
mesh = pyvista.read("mesh.obj")
print(mesh)
mesh.plot()

PolyData (0x10c41ffea60)
  N Cells:    24459
  N Points:   97836
  N Strips:   0
  X Bounds:   -5.843e+00, 5.843e+00
  Y Bounds:   -5.660e-02, 2.068e+01
  Z Bounds:   -1.853e+00, 1.917e+00
  N Arrays:   5


Widget(value="<iframe src='http://localhost:51167/index.html?ui=P_0x10c41fc4a90_0&reconnect=auto' style='width…

In [35]:
scaledMesh = mesh.rotate_x(-90)
scaledMesh = putModelonBoard(scaledMesh, scale=0.2)
print(scaledMesh)
scaledMesh.plot()

PolyData (0x10c46c7f9a0)
  N Cells:    24459
  N Points:   97836
  N Strips:   0
  X Bounds:   0.000e+00, 2.337e+00
  Y Bounds:   0.000e+00, 7.540e-01
  Z Bounds:   0.000e+00, 4.148e+00
  N Arrays:   5


Widget(value="<iframe src='http://localhost:51167/index.html?ui=P_0x10c427cffa0_1&reconnect=auto' style='width…

In [36]:
for i, fname in enumerate(foundImages):
    img = cv2.imread(os.path.join(selectedPath, fname))
    img = undistort(img, mtx, dist)
    imPoints, _ = cv2.projectPoints(scaledMesh.points, rvecs[i], tvecs[i], mtx, dist)
    imPoints = np.squeeze(imPoints)
    xx = projectCircle(img, imPoints)
    cv2.imwrite(os.path.join("projected", fname), xx)

In [37]:
def concat_vh(list_2d):
    # return final image
    return cv2.vconcat([cv2.hconcat(list_h) 
                        for list_h in list_2d])

In [38]:
chessboards = []
# image resizing
for i, fname in enumerate(foundImages):
    img = cv2.imread(os.path.join("chessboard", fname))
    chessboards.append(img)

imgs = concat_vh([[chessboards[0], chessboards[1], chessboards[2]],
                      [chessboards[3], chessboards[4], chessboards[5]]])
# show the output image
cv2.imwrite('chessboards.jpg', imgs)

True

In [39]:
undistorteds = []
# image resizing
for i, fname in enumerate(foundImages):
    img = cv2.imread(os.path.join("concated", fname))
    undistorteds.append(img)

imgs = concat_vh([[undistorteds[0], undistorteds[1], undistorteds[2]],
                      [undistorteds[3], undistorteds[4], undistorteds[5]]])
# show the output image
cv2.imwrite('undistorteds.jpg', imgs)

True

In [40]:
projecteds = []
# image resizing
for i, fname in enumerate(foundImages):
    img = cv2.imread(os.path.join("projected", fname))
    projecteds.append(img)

imgs = concat_vh([[projecteds[0], projecteds[1], projecteds[2]],
                      [projecteds[3], projecteds[4], projecteds[5]]])
# show the output image
cv2.imwrite('projecteds.jpg', imgs)

True

#### Task 3

In [41]:
def skew(x):
    return np.array([[0, -x[2], x[1]],
                     [x[2], 0, -x[0]],
                     [-x[1], x[0], 0]])

In [42]:
def normalize2Dpts(A: np.ndarray) -> np.ndarray:
    """
    Finds similarity for normalization of 2D homography data points with zero mean and sqrt(2) distance to origin.
        
        Parameters:
                A   (numpy.ndarray): Input data points.
        Returns:
                Transformed points and similarity transform matrix.
    """
    # Translation
    mean = np.mean(A, axis=0)
    shifted = A - mean
    T1 = np.array([[1,      0,  -mean[0]],
                   [0,      1,  -mean[1]],
                   [0,      0,         1]], dtype=np.float32)
    for i in range(A.shape[0]):
        shifted[i] = T1@A[i]

    # Scale
    dSum = np.float32(0)
    for i in range(shifted.shape[0]):
        dSum += np.sqrt(shifted[i][0]**2 + shifted[i][1]**2)
    scale = np.sqrt(2)*shifted.shape[0]/dSum
    T2 = np.array([[scale, 0,        0],
                  [0,        scale,  0],
                  [0,        0,      1]], dtype=np.float32)
    normalized = np.zeros_like(A, dtype=np.float32)
    for i in range(shifted.shape[0]):
        normalized[i] = T2@shifted[i]

    T = T2@T1

    return normalized, T

In [43]:
def EightPointAlgorithm(x, x_hat):
    assert x.shape == x_hat.shape, "shapes do not match"
    N = x.shape[0]
    assert N >= 8, "minimum 8 correspondences are required"

    # Normalize points with 0 mean and sqrt2 variance
    x_norm, xNormalizer = normalize2Dpts(x)
    x_hat_norm, x_hatNormalizer = normalize2Dpts(x_hat)

    # Convert into linear system and solve with SVD
    E_s = np.zeros(shape=(9))
    A = np.zeros(shape=(N,9))

    for i in range(N):
        A[i] = np.kron(x_norm[i], x_hat_norm[i])
    
    u, sigma, vh = np.linalg.svd(A)
    
    E_s = (vh.T)[:,-1]
    E_hat = E_s.reshape(3,3)
    # E_hat = np.array([[E_s[0], E_s[1], E_s[2]],
    #                   [E_s[3], E_s[4], E_s[5]],
    #                   [E_s[6], E_s[7], E_s[8]]])

    # Project estimated matrix onto essential space
    u, sigma, vh = np.linalg.svd(E_hat)
    sigma = np.diag(sigma)

    newSigma = copy.copy(sigma)
    newSigma[0,0] = (sigma[0,0] + sigma[1,1])/2
    newSigma[1,1] = (sigma[0,0] + sigma[1,1])/2
    newSigma[2,2] = 0

    E_tilda = u@newSigma@vh

    # Denormalize
    E_tilda = (x_hatNormalizer.T)@E_tilda@xNormalizer

    return E_tilda


In [44]:
def decomposeEssential(E, K1, K2, x, x_hat):
    # Decompose T and R
    u, sigma, vh = np.linalg.svd(E)
    u = -u if np.linalg.det(u) < 0 else u
    v = vh.T
    v = -v if np.linalg.det(v) < 0 else v
    vh = v.T

    W = np.array([[0,-1,0], [1,0,0], [0,0,1]])
    R2 = u@W@vh
    T2 = u[:,2]
    
    M = skew((R2.T)@T2)
    X1 = M@np.linalg.inv(K1)@x[0]
    X2 = M@(R2.T)@np.linalg.inv(K2)@x_hat[0]
    if(X1[2]*X2[2] < 0):
        R2 = u@(W.T)@vh
        M = skew((R2.T)@T2)
        X1 = M@np.linalg.inv(K1)@x[0]
    
    if(X1[2] < 0):
        T2 = -T2
    
    P1 = K1 @ np.concatenate((np.eye(3), np.array([[0, 0, 0]]).T), axis=1)
    P2 = K2 @ np.concatenate((R2, T2.reshape((3, 1))), axis=1)

    return P1, P2

In [45]:
# This function are coded by utilizing "3D Vision Stereo Epipolar Geometry.pdf"
def triangulate(M1, M2, x, x_hat):
    N = x.shape[0]
    X = np.zeros(shape=(N, 3))
    
    for i in range(N):
        A = np.zeros(shape=(4,4))

        A[0] = x[i,0]*M1[2] - M1[0]
        A[1] = x[i,1]*M1[2] - M1[1]
        
        A[2] = x_hat[i,0]*M2[2] - M2[0]
        A[3] = x_hat[i,1]*M2[2] - M2[1]

        u, sigma, vh = np.linalg.svd(A)
        X_estimate = np.transpose(vh)[:,-1]
        X_estimate = X_estimate/X_estimate[-1]      
        X[i] = X_estimate[:3]

    return X

In [46]:
im1 = cv2.imread("1.JPG")
im2 = cv2.imread("2.JPG")

camMat = scipy.io.loadmat("cube_data.mat")

K = camMat["Calib"]
x = camMat["x1"].T
x_hat = camMat["x2"].T
# undistort
xk = x @ np.linalg.inv(K).T             # In camera coordinate
x_hatk = x_hat @ np.linalg.inv(K).T     # In camera coordinate

im1Circled = copy.copy(im1)
im2Circled = copy.copy(im2)
for i in range(x.shape[0]):
    im1Circled = cv2.circle(im1Circled, (int(x[i,0]), int(x[i,1])), 3, [0, 255, 0], thickness=-1)
    im2Circled = cv2.circle(im2Circled, (int(x_hat[i,0]), int(x_hat[i,1])), 3, [0, 255, 0], thickness=-1)
cv2.imwrite("1Circled.JPG", im1Circled)
cv2.imwrite("2Circled.JPG", im2Circled)

True

In [49]:
# Calculate essential matrix by using camera coordinates
E = EightPointAlgorithm(xk, x_hatk)
# Decompose essential matrix to R and T by using pixel coordinates, and return camera matrix
P1, P2 = decomposeEssential(E, K, K, x, x_hat)
# Triangulate back to 3D points
X = triangulate(P1, P2, x_hat, x)

# Plot the result
%matplotlib 
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter(X[:,0], X[:,1], X[:,2], facecolors='none', edgecolors='blue')

Using matplotlib backend: TkAgg


In [50]:
%matplotlib 
fig = plt.figure()
ax = plt.axes()
ax.scatter(xk[:,0], xk[:,1], facecolors='none', edgecolors='blue')

Using matplotlib backend: TkAgg
